**Корректность проверена на Python 3.7:**
+ pandas 0.23.0
+ numpy 1.14.5
+ scipy 1.1.0

In [5]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/stepik/module4/w3. dependencies'
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/stepik/module4/w3. dependencies
'3.5 multiple tests.ipynb'   foodmart.products.tsv   water.ipynb
'3.6.3 auc.ipynb'	     foodmart.sales.tsv      water.txt
 AUCs.txt		     illiteracy.ipynb
 correlation.ipynb	     illiteracy.txt


# Множественная проверка гипотез

In [2]:
import pandas as pd
import numpy as np

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import scipy
print(np.__version__)
print(pd.__version__)
print(scipy.__version__)

1.19.5
1.1.5
1.4.1


## Foodmart product sales 

In [6]:
sales = pd.read_csv('foodmart.sales.tsv', sep = '\t', header = 0, parse_dates = [2])

In [7]:
sales.head()

,product_id,store_id,date,sales
0,4,6,1997-01-01,4
1,25,6,1997-01-01,3
2,48,6,1997-01-01,3
3,76,6,1997-01-01,4
4,119,6,1997-01-01,3


In [8]:
products = pd.read_csv('foodmart.products.tsv', sep = '\t', header = 0)

In [9]:
products.head()

,product_class_id,product_id,brand_name,product_name,SKU,SRP,gross_weight,net_weight,recyclable_package,low_fat,units_per_case,cases_per_pallet,shelf_width,shelf_height,shelf_depth
0,30,1,Washington,Washington Berry Juice,90748583674,2.85,8.39,6.39,False,False,30,14,16.9,12.60,7.40
1,52,2,Washington,Washington Mango Drink,96516502499,0.74,7.42,4.42,False,True,18,8,13.4,3.71,22.60
2,52,3,Washington,Washington Strawberry Drink,58427771925,0.83,13.10,11.10,True,True,17,13,14.4,11.00,7.77
3,19,4,Washington,Washington Cream Soda,64412155747,3.64,10.60,9.60,True,False,26,10,22.9,18.90,7.93
4,19,5,Washington,Washington Diet Soda,85561191439,2.19,6.66,4.65,True,False,7,10,20.7,21.90,19.20


In [10]:
sales = sales.merge(products[['product_id', 'product_name']], 
                    on = ['product_id'], how = 'inner')

In [11]:
sales.head()

,product_id,store_id,date,sales,product_name
0,4,6,1997-01-01,4,Washington Cream Soda
1,4,7,1997-01-05,3,Washington Cream Soda
2,4,6,1997-01-06,2,Washington Cream Soda
3,4,17,1997-01-11,2,Washington Cream Soda
4,4,24,1997-01-11,2,Washington Cream Soda


## Корреляция Пирсона

In [12]:
sparse_sales = pd.pivot_table(sales, values='sales', index=['date', 'store_id'],
                     columns=['product_name'], fill_value = 0)

In [13]:
sparse_sales.head()

product_name         ADJ Rosy Sunglasses  ...  Washington Strawberry Drink
date       store_id                       ...                             
1997-01-01 6                           0  ...                            0
           14                          0  ...                            0
1997-01-02 11                          0  ...                            0
           23                          0  ...                            0
1997-01-03 7                           0  ...                            0

[5 rows x 1559 columns]

In [14]:
%%time 
corr_data = []

for i, lhs_column in enumerate(sparse_sales.columns):
    for j, rhs_column in enumerate(sparse_sales.columns):
        if i >= j:
            continue
        
        corr, p = pearsonr(sparse_sales[lhs_column], sparse_sales[rhs_column])
        corr_data.append([lhs_column, rhs_column, corr, p])

CPU times: user 2min 33s, sys: 732 ms, total: 2min 33s
Wall time: 2min 34s


In [ ]:
sales_correlation = pd.DataFrame.from_records(corr_data)
sales_correlation.columns = ['product_A', 'product_B', 'corr', 'p']

In [ ]:
sales_correlation.head()

,product_A,product_B,corr,p
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484


Сколько гипотез об отсутствии корреляции отвергается без поправки на множественную проверку?

In [ ]:
(sales_correlation.p < 0.05).value_counts()

False    982453
True     232008
Name: p, dtype: int64

## Поправка на множественную проверку

### Метод Холма

In [ ]:
reject, p_corrected, a1, a2 = multipletests(sales_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [ ]:
sales_correlation['p_corrected'] = p_corrected
sales_correlation['reject'] = reject

In [ ]:
sales_correlation.head()

,product_A,product_B,corr,p,p_corrected,reject
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414,1.0,False
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396,1.0,False
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546,1.0,False
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529,1.0,False
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484,1.0,False


In [ ]:
sales_correlation.reject.value_counts()

False    1212733
True        1728
Name: reject, dtype: int64

In [ ]:
sales_correlation[sales_correlation.reject == True].sort_values(by='corr', ascending=False)

,product_A,product_B,corr,p,p_corrected,reject
1063670,Just Right Vegetable Soup,Plato French Roast Coffee,0.340598,1.226033e-22,1.488970e-16,True
885574,Great Muffins,Nationeel Grape Fruit Roll,0.322176,2.688803e-20,3.265443e-14,True
473067,Club Low Fat Cottage Cheese,Skinner Strawberry Drink,0.306701,1.883995e-18,2.288034e-12,True
1181001,Robust Monthly Home Magazine,Tri-State Lemons,0.303269,4.674973e-18,5.677558e-12,True
1160248,Pleasant Regular Ramen Soup,Shady Lake Ravioli,0.298502,1.619119e-17,1.966350e-11,True
885261,Great Muffins,High Quality Copper Cleaner,0.297160,2.287551e-17,2.778130e-11,True
1151585,Plato Salt,Tri-State Onions,0.291224,1.032464e-16,1.253881e-10,True
1092548,Mighty Good Monthly Home Magazine,Tell Tale Baby Onion,0.290458,1.250831e-16,1.519077e-10,True
678154,Even Better Buttermilk,Sunset Toilet Paper,0.290046,1.386446e-16,1.683774e-10,True
914945,Hermanos Macintosh Apples,Plato French Roast Coffee,0.289095,1.757608e-16,2.134531e-10,True


### Метод Бенджамини-Хохберга

In [ ]:
reject, p_corrected, a1, a2 = multipletests(sales_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [ ]:
sales_correlation['p_corrected'] = p_corrected
sales_correlation['reject'] = reject

In [ ]:
sales_correlation.head()

In [ ]:
sales_correlation.reject.value_counts()

False    1138407
True       76054
Name: reject, dtype: int64

In [ ]:
sales_correlation[sales_correlation.reject == True].sort_values(by='corr', ascending=False)